In [236]:
#Imports 
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from scipy import sparse
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import random

- The paper did not mention how they have calculated their clustering accuracy. Hence we decided to use this function from scikit-learn.
- All these clustering evaluation metrics have a maximum value of 1.0 - hence higher the value the better


In [71]:
from collections import defaultdict
from sklearn import metrics
from time import time

evaluations = []
evaluations_std = []


def fit_and_evaluate(km, X, name=None, n_runs=5):
    name = km.__class__.__name__ if name is None else name

    train_times = []
    scores = defaultdict(list)
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        train_times.append(time() - t0)
        scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(labels, km.labels_)
        )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
    train_times = np.asarray(train_times)

    print(f"clustering done in {train_times.mean():.2f} ± {train_times.std():.2f} s ")
    evaluation = {
        "estimator": name,
        "train_time": train_times.mean(),
    }
    evaluation_std = {
        "estimator": name,
        "train_time": train_times.std(),
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} ± {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)

- This function is used to sample documenst from the corpus to create the desired dataset

In [212]:
def sample_docs(dataset, doc_num, curr_file_nums):
  final_data = []
  target_vals = []
  while(curr_file_nums != doc_num):
    for i in range(len(dataset.target)):
      if(curr_file_nums[dataset.target[i]] < doc_num[dataset.target[i]]):
        curr_file_nums[dataset.target[i]] = curr_file_nums[dataset.target[i]] + 1
        final_data.append(dataset.data[i])
        target_vals.append(dataset.target[i])
        
  return final_data, target_vals

##Objective of this experiment: 
- In the paper they prove that performing k-means clustering in Principal component subspace helps improve clustering accuracy.

- It shows that PCA is actually beneficial for K means clustering 

`performing the same experimanet on 60 datasets becomes repetitive - hence we have shown these results on A5(balanced and A5(unbalanced) datasets`

In [225]:
categories = [
    "comp.graphics",
    "rec.motorcycles",
    "rec.sport.baseball",
    "sci.space",
    "talk.politics.mideast"
]

dataset = fetch_20newsgroups(
    remove=("headers", "footers", "quotes"),
    subset="all",
    categories = categories,
    shuffle = True,
    random_state = 42,
)

print(f"the datatype of dataset is{type(dataset)} and it contains the following attributes {dataset.keys()}")
labels = dataset.target
unique_labels = np.unique(dataset.target)
true_k = unique_labels.shape[0]

print(f"We found {len(dataset.data)} documents - belonging to the {true_k} specified categories namely {dataset.target_names}")

the datatype of dataset is<class 'sklearn.utils.Bunch'> and it contains the following attributes dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])
We found 4890 documents - belonging to the 5 specified categories namely ['comp.graphics', 'rec.motorcycles', 'rec.sport.baseball', 'sci.space', 'talk.politics.mideast']


In [239]:
np.unique(dataset.target)
unique, counts = np.unique(dataset.target, return_counts=True)

doc_freq = dict(zip(unique, counts))
doc_freq

print(f"Number of documents belonging to {dataset.target_names[0]}: {doc_freq[0]}")
print(f"Number of documents belonging to {dataset.target_names[1]}: {doc_freq[1]}") 
print(f"Number of documents belonging to {dataset.target_names[2]}: {doc_freq[2]}") 
print(f"Number of documents belonging to {dataset.target_names[3]}: {doc_freq[3]}") 
print(f"Number of documents belonging to {dataset.target_names[4]}: {doc_freq[4]}") 

Number of documents belonging to comp.graphics: 973
Number of documents belonging to rec.motorcycles: 996
Number of documents belonging to rec.sport.baseball: 994
Number of documents belonging to sci.space: 987
Number of documents belonging to talk.politics.mideast: 940


In [174]:
dataset.data[1]

'\n\nFirst, you seem to assume all atheists think alike.  An atheist does not\nbelieve in the existence of a god.  Our opinions on issues such as \ncapital punishment and abortion, however, vary greatly.  \n\nIf you were attacking the views of a particular atheist (Benedikt, I \npresume), then please present your argument as such and do not lump us\nall together.\n\nAs for the issues, let\'s start with abortion.  Personally, I do not support\nabortion as a means of population control or contraception-after-the-fact.\nHowever, I support the right of any woman to have an abortion, regardless\nof what my personal views may be, because it would be arrogant of me to tell\nany individual what he/she may or may not do to his/her body, and the domain\nof legislators should not extend into the uterus.  That\'s my opinion, and I\nam sure many atheists and theists would disagree with me.\n\nI do not defend homosexuality as a means of population control, but I \ncertainly defend it as an end to it

-----------------
#Balanced A5 dataset

Sample equal number of documents from the dataset (100 in this case)

In [240]:
(A5_dataset_bal, labels) = sample_docs(dataset, [100,100,100,100,100], [0,0,0,0,0])

In [241]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
    stop_words="english",
    max_features=1000
)
X_tfidf = vectorizer.fit_transform(A5_dataset_bal)

print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

n_samples: 500, n_features: 1000


- There exists an issue with scikit-learn pca implementation which requires the input to be a dense matrix. This is problematic for us as TfidfVectorizer returns a sparse result

- following some of the comments in this github page, we decided to centre the data and then apply truncated SVD which achieves a similar result as PCA.

the issue: https://github.com/scikit-learn/scikit-learn/issues/12794 

In [243]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

def dim_reduction_pca(TFIDF, final_dim):
  
  #very important
  temp = TFIDF.toarray()
  mean = temp.mean(axis = 0)
  centered_tfidf = temp - mean

  pca = make_pipeline(TruncatedSVD(n_components=100), Normalizer(copy=False))
  new_tfidf = pca.fit_transform(centered_tfidf)
  return new_tfidf

In [244]:
X_tfidf.shape

(500, 1000)

In [246]:
kmeans = KMeans(
    n_clusters = 5,
    max_iter = 20,
    n_init = 5,
)
print(f"K means clustering with {X_tfidf.shape[1]} words")
fit_and_evaluate(kmeans, X_tfidf, name="KMeans\non tf-idf vectors")

K means clustering with 1000 words
clustering done in 0.07 ± 0.01 s 
Homogeneity: 0.313 ± 0.066
Completeness: 0.359 ± 0.074
V-measure: 0.334 ± 0.069
Adjusted Rand-Index: 0.219 ± 0.059
Silhouette Coefficient: 0.009 ± 0.004


In [249]:
#Dimension reduction
new_tfidf = dim_reduction_pca(X_tfidf, 100)

In [250]:
kmeans = KMeans(
    n_clusters= 5,
    max_iter = 20,
    n_init = 10,
)
print(f"K means clustering with {new_tfidf.shape[1]} words After PCA")
fit_and_evaluate(kmeans, new_tfidf, name="KMeans\non tf-idf vectors")

K means clustering with 100 words After PCA
clustering done in 0.37 ± 0.08 s 
Homogeneity: 0.381 ± 0.037
Completeness: 0.454 ± 0.035
V-measure: 0.414 ± 0.037
Adjusted Rand-Index: 0.328 ± 0.035
Silhouette Coefficient: 0.036 ± 0.001


`It is clearly seen that there is an increase in the clustering accuracy `

------------------
#Unbalanced A5 dataset

Sample unequal number of documents from the dataset for each news group

In [252]:
(A5_dataset_unbal, labels) = sample_docs(dataset, [200,140,120,100,60], [0,0,0,0,0])

In [255]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
    stop_words="english",
    max_features=1000
)
X_tfidf = vectorizer.fit_transform(A5_dataset_unbal)

print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

n_samples: 620, n_features: 1000


In [256]:
X_tfidf.shape

(620, 1000)

In [257]:
kmeans = KMeans(
    n_clusters = 5,
    max_iter = 20,
    n_init = 5,
)
print(f"K means clustering with {X_tfidf.shape[1]} words")
fit_and_evaluate(kmeans, X_tfidf, name="KMeans\non tf-idf vectors")

K means clustering with 1000 words
clustering done in 0.08 ± 0.02 s 
Homogeneity: 0.298 ± 0.049
Completeness: 0.307 ± 0.040
V-measure: 0.302 ± 0.044
Adjusted Rand-Index: 0.224 ± 0.063
Silhouette Coefficient: 0.005 ± 0.003


In [258]:
#Dimension reduction
new_tfidf = dim_reduction_pca(X_tfidf, 100)

In [259]:
kmeans = KMeans(
    n_clusters= 5,
    max_iter = 20,
    n_init = 10,
)
print(f"K means clustering with {new_tfidf.shape[1]} words After PCA")
fit_and_evaluate(kmeans, new_tfidf, name="KMeans\non tf-idf vectors")

K means clustering with 100 words After PCA
clustering done in 0.53 ± 0.15 s 
Homogeneity: 0.369 ± 0.053
Completeness: 0.404 ± 0.063
V-measure: 0.386 ± 0.058
Adjusted Rand-Index: 0.312 ± 0.047
Silhouette Coefficient: 0.032 ± 0.001


`Once again the clustering accuracy is shown to increase`